In [95]:
using DataFrames, CSV, DelimitedFiles, Plots, FFTW
include("/home/kirk/Documents/research/CHANDRATest/DJM.jl")

Main.DJM

In [2]:
function genFreqSums(powMatrix)
    sums=zeros(length(powMatrix[1,1,2:end]))
    for i in 1:length(powMatrix[1,1,2:end])
        sums[i]+=sum(powMatrix[:,:,i+1]) #i+1 becaus normal matrix is in 1st slice
    end
    return sums
end
function sumsLog(sums)
    sums4log=zeros(length(sums))
    for i in 1:length(sums4log)
        if sums[i]==0
            sums4log[i]=1
        else
            sums4log[i]=sums[i]
        end
    end
    return sums4log
end
function genFRange(df,powList)
    fbins=ceil(maximum(powList.freq[:])/df)
    frange=zeros(Int(fbins))
    for i in 1:length(frange)
        frange[i]=i*df
    end
    return frange
end

genFRange (generic function with 1 method)

In [96]:
pows8=CSV.read("cxc.fftinfo.all8")
pows8MatrixOVERLAP,pows8src=DJM.createPowMatrix(pows8,1)

1255743 potential matches had the same obsid and were excluded.


([0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 6.0]

[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

...

[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], Any[Tuple{Int64,Int64}[(65, 508)], Tuple{Int64,Int64}[(65, 509)], Tuple{Int64,Int64}[(65, 510)], Tuple{Int64,Int64}[(66, 511)], Tuple{Int64,Int64}[(66, 512)], Tuple{Int64,Int64}[(124, 1339)], Tuple{Int64,Int64}[(124, 1340)], Tuple{Int64,Int64}[(124, 1341)], Tuple{Int64,Int64}[(125, 1339)], Tuple{Int64,Int64}[(125, 1340)]  …  Tuple{Int64,Int64}[(10985, 11045)], Tuple{Int64,Int64}[(10985, 11046)], Tuple{Int64,Int64}[(10986, 11043)], Tuple{Int64,Int64}[(10986, 11044)], Tuple{Int64,Int64}[(10986, 1104

In [97]:
pows8src

250-element Array{Any,1}:
 Tuple{Int64,Int64}[(65, 508)]     
 Tuple{Int64,Int64}[(65, 509)]     
 Tuple{Int64,Int64}[(65, 510)]     
 Tuple{Int64,Int64}[(66, 511)]     
 Tuple{Int64,Int64}[(66, 512)]     
 Tuple{Int64,Int64}[(124, 1339)]   
 Tuple{Int64,Int64}[(124, 1340)]   
 Tuple{Int64,Int64}[(124, 1341)]   
 Tuple{Int64,Int64}[(125, 1339)]   
 Tuple{Int64,Int64}[(125, 1340)]   
 Tuple{Int64,Int64}[(125, 1341)]   
 Tuple{Int64,Int64}[(325, 1250)]   
 Tuple{Int64,Int64}[(325, 1251)]   
 ⋮                                 
 Tuple{Int64,Int64}[(10985, 11043)]
 Tuple{Int64,Int64}[(10985, 11044)]
 Tuple{Int64,Int64}[(10985, 11045)]
 Tuple{Int64,Int64}[(10985, 11046)]
 Tuple{Int64,Int64}[(10986, 11043)]
 Tuple{Int64,Int64}[(10986, 11044)]
 Tuple{Int64,Int64}[(10986, 11045)]
 Tuple{Int64,Int64}[(10986, 11046)]
 Tuple{Int64,Int64}[(10987, 11048)]
 Tuple{Int64,Int64}[(10988, 11048)]
 Tuple{Int64,Int64}[(10989, 11048)]
 Tuple{Int64,Int64}[(11010, 11077)]

In [4]:
MCList=CSV.read("MCTestList.csv")
noMissing=dropmissing(MCList,:pulse_period,disallowmissing=true)

,name,ra,dec,porb,fx_limit,fx,fx_max
,String,Float64⍰,Float64⍰,Float64⍰,String⍰,Float64⍰,Float64⍰
1,XTE SMC144s,missing,missing,61.2,missing,missing,missing
2,H 0053-739,13.6508,-73.6764,missing,<,1.0,50.0
3,RX J0117.6-7330,19.4225,-73.5136,missing,missing,10.0,missing
4,2U 0115-737,19.2729,-73.4422,3.89,missing,0.5,57.0
5,AX J0057.4-7325,14.3617,-73.4172,missing,missing,0.01,0.11
6,RX J0049.7-7323,12.425,-73.3875,394.0,missing,0.001,2.0
7,RX J0052.1-7319,13.0579,-73.3203,missing,missing,0.024,4.7
8,XTE SMC46,12.75,-73.3,missing,~,0.36,missing
9,XTE J0111.2-7317,17.785,-73.2794,missing,missing,95.7,missing


In [5]:
justFreq8=pows8.freq[:]
knownFreq=noMissing.pulse_period[:].^-1
using Formatting  
function findFreqMatches(knownList,freqList,srcIndList)
    confirmedFreqInd=[] #keep track of places where frequencies potentially match to known pulsars
    unconfirmedFreqInd=[]
    
    for i in 1:length(srcIndList)
        print(format(i/length(srcIndList)*100,precision=2),"% complete\r") #output % tracker
        freq1Ind=first.(srcIndList[i])[1] #first element from tuple at index i, contains index where freq is
        freq2Ind=last.(srcIndList[i])[1] #1 needed because technically last.(A) returns [A] not just A
        freq1=freqList[freq1Ind]
        freq2=freqList[freq2Ind]
        if freq1>freq2 #check which one is bigger for range argument later
            freqMax=freq1
            freqMin=freq2
        else
            freqMax=freq2
            freqMin=freq1
        end
        
        for knownFreq in knownList
            if freqMax>knownFreq && freqMin<knownFreq
                push!(confirmedFreqInd,srcIndList[i]) #push the tuple of indices to confirmed list
            end
        end
    end
    
    #for ind in srcIndList
    #    if ind in confirmedFreqInd
    #        continue
    #    else
    #        push!(unconfirmedFreqInd, ind) #if we didn't already put tuple in confirmed list put it in this one
    #    end
    #end
    
    return confirmedFreqInd,unconfirmedFreqInd
end
                
    

findFreqMatches (generic function with 1 method)

In [6]:
confirmedTest,unconfirmedTest=findFreqMatches(knownFreq,justFreq8,pows8src)

Excessive output truncated after 524289 bytes.

(Any[Tuple{Int64,Int64}[(3, 4)], Tuple{Int64,Int64}[(3, 4)], Tuple{Int64,Int64}[(3, 4)], Tuple{Int64,Int64}[(3, 4)], Tuple{Int64,Int64}[(3, 4)], Tuple{Int64,Int64}[(3, 4)], Tuple{Int64,Int64}[(3, 4)], Tuple{Int64,Int64}[(3, 4)], Tuple{Int64,Int64}[(3, 4)], Tuple{Int64,Int64}[(3, 4)]  …  Tuple{Int64,Int64}[(11253, 11255)], Tuple{Int64,Int64}[(11253, 11255)], Tuple{Int64,Int64}[(11253, 11255)], Tuple{Int64,Int64}[(11253, 11255)], Tuple{Int64,Int64}[(11253, 11255)], Tuple{Int64,Int64}[(11253, 11255)], Tuple{Int64,Int64}[(11254, 11255)], Tuple{Int64,Int64}[(11254, 11255)], Tuple{Int64,Int64}[(11254, 11255)], Tuple{Int64,Int64}[(11254, 11255)]], Any[])

In [7]:
confirmedTest

986681-element Array{Any,1}:
 Tuple{Int64,Int64}[(3, 4)]        
 Tuple{Int64,Int64}[(3, 4)]        
 Tuple{Int64,Int64}[(3, 4)]        
 Tuple{Int64,Int64}[(3, 4)]        
 Tuple{Int64,Int64}[(3, 4)]        
 Tuple{Int64,Int64}[(3, 4)]        
 Tuple{Int64,Int64}[(3, 4)]        
 Tuple{Int64,Int64}[(3, 4)]        
 Tuple{Int64,Int64}[(3, 4)]        
 Tuple{Int64,Int64}[(3, 4)]        
 Tuple{Int64,Int64}[(3, 4)]        
 Tuple{Int64,Int64}[(3, 4)]        
 Tuple{Int64,Int64}[(3, 4)]        
 ⋮                                 
 Tuple{Int64,Int64}[(11253, 11254)]
 Tuple{Int64,Int64}[(11253, 11254)]
 Tuple{Int64,Int64}[(11253, 11255)]
 Tuple{Int64,Int64}[(11253, 11255)]
 Tuple{Int64,Int64}[(11253, 11255)]
 Tuple{Int64,Int64}[(11253, 11255)]
 Tuple{Int64,Int64}[(11253, 11255)]
 Tuple{Int64,Int64}[(11253, 11255)]
 Tuple{Int64,Int64}[(11254, 11255)]
 Tuple{Int64,Int64}[(11254, 11255)]
 Tuple{Int64,Int64}[(11254, 11255)]
 Tuple{Int64,Int64}[(11254, 11255)]

In [9]:
test1=first.(pows8src[1])[1]
test2=last.(pows8src[1])[1]

4

In [12]:
freq1=justFreq8[test1]

0.01854539992678998

In [13]:
freq2=justFreq8[test2]

0.14409611624532392

In [14]:
delta=abs(freq2-freq1)

0.12555071631853396

In [16]:
raTest1=pows8.ra[test1]
raTest2=pows8.ra[test2]
decTest1=pows8.dec[test1]
decTest2=pows8.dec[test2]

-73.39613260135003

In [17]:
raDel=abs(raTest2-raTest1)

0.0

In [18]:
decDel=abs(decTest2-decTest1)

0.0

In [23]:
#from other notebook, just eyeballing decided to test around f=0.107
df=0.003 #roughly maximum range surrounding spike without any other spikes
freq=0.107
matchIndList=[]
for i=1:length(knownFreq)
    freqCheck=knownFreq[i]
    freqMin=freq-df
    freqMax=freq+df
    if freqCheck>freqMin && freqCheck<freqMax
        push!(matchIndList,i) #appends index match found at
    end
end

In [24]:
matchIndList

1-element Array{Any,1}:
 15

In [25]:
noMissing[15,:]

,name,ra,dec,porb,fx_limit,fx,fx_max,pulse_period
,String,Float64⍰,Float64⍰,Float64⍰,String⍰,Float64⍰,Float64⍰,Float64
15,RX J0049.2-7311,12.3077,-73.1935,missing,missing,0.04,missing,9.1321


In [26]:
noMissing.pulse_period[15]^-1

0.10950383810952574

In [27]:
matchRA,matchDEC=noMissing.ra[15],noMissing.dec[15]

(12.30767, -73.19353)

In [70]:
fDel=[]
for i=2:2:length(pows8src)
    push!(fDel,pows8src[i])
end

In [71]:
fDel

122761-element Array{Any,1}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮  
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [66]:
fAvgList=[]
for i=2:2:length(pows8src)
    push!(fAvgList,pows8src[i])
end

In [67]:
fAvgList

122761-element Array{Any,1}:
 0.08132075808605695  
 0.000553511162953775 
 0.0005540791579901564
 0.0011879219847346894
 0.001187312481407478 
 0.0005543635928761885
 0.0007120221619271306
 0.0007911357354745896
 0.0005540791579901564
 0.0016377013632435772
 0.0011385348222697047
 0.0018216557156315275
 0.002618630091220321 
 ⋮                    
 0.07068182345318846  
 0.05814382771557591  
 0.07836698343048767  
 0.06501717501921675  
 0.06528777924376955  
 0.08551093495868131  
 0.06772321726474465  
 0.0879463729796564   
 0.0882169772042092   
 0.07591420884170297  
 0.11299109645393082  
 0.116907163071889    

In [62]:
pows8

,obsid,srcnum,startdate,exptime,ra,dec,freq,power
,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Float64
1,1026,26,51833.8,6458.48,19.9382,-73.4501,0.00123975,13.2664
2,1026,8,51833.8,6458.48,19.5522,-73.6193,0.154427,9.16655
3,1027,107,51855.0,6133.27,18.8667,-73.3961,0.0185454,9.14431
4,1027,107,51855.0,6133.27,18.8667,-73.3961,0.144096,8.02378
5,1027,36,51855.0,6133.27,19.782,-73.5006,0.0813522,8.00401
6,1027,40,51855.0,6133.27,19.7033,-73.4917,0.0417401,8.04439
7,1027,54,51855.0,6133.27,19.5056,-73.4696,0.14484,8.14304
8,1027,58,51855.0,6133.27,19.4489,-73.4632,0.0598763,8.05626
9,1027,91,51855.0,6133.27,19.0827,-73.4213,0.0622972,8.75528


In [65]:
pows8src

245522-element Array{Any,1}:
  Tuple{Int64,Int64}[(3, 4)]        
 0.08132075808605695                
  Tuple{Int64,Int64}[(13, 14)]      
 0.000553511162953775               
  Tuple{Int64,Int64}[(15, 16)]      
 0.0005540791579901564              
  Tuple{Int64,Int64}[(18, 19)]      
 0.0011879219847346894              
  Tuple{Int64,Int64}[(23, 24)]      
 0.001187312481407478               
  Tuple{Int64,Int64}[(28, 29)]      
 0.0005543635928761885              
  Tuple{Int64,Int64}[(32, 34)]      
 ⋮                                  
  Tuple{Int64,Int64}[(11250, 11251)]
 0.06772321726474465                
  Tuple{Int64,Int64}[(11250, 11252)]
 0.0879463729796564                 
  Tuple{Int64,Int64}[(11251, 11252)]
 0.0882169772042092                 
  Tuple{Int64,Int64}[(11253, 11254)]
 0.07591420884170297                
  Tuple{Int64,Int64}[(11253, 11255)]
 0.11299109645393082                
  Tuple{Int64,Int64}[(11254, 11255)]
 0.116907163071889                  

# TEST FOR ONE PAIR BELOW

In [73]:
identifiedPair=[3,4]
rdist=DJM.gcdist(pows8.ra[3],pows8.dec[3],pows8.ra[4],pows8.dec[4])

0.0

In [75]:
fAvg=(pows8.freq[3]+pows8.freq[4])/2
println("avg pow: ",fAvg)
println("pow 1: ",pows8.freq[3])
println("pow 2: ",pows8.freq[4])

avg pow: 0.08132075808605695
pow 1: 0.01854539992678998
pow 2: 0.14409611624532392


In [76]:
dt=(pows8.startdate[3]-pows8.startdate[4])*24*3600

0.0

In [87]:
#CHECKING IF ALL DT = 0
dtList=[]
for i=1:2:(length(pows8src)-1)
    pair=pows8src[i]
    ind1=first.(pair)[1]
    ind2=last.(pair)[1]
    dt=abs((pows8.startdate[ind1]-pows8.startdate[ind2])*24*3600)
    push!(dtList,dt)
end
    

In [88]:
dtList

122761-element Array{Any,1}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮  
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [89]:
using Statistics
println("std: ",std(dtList))
println("mean: ",mean(dtList))
println("min: ",minimum(dtList))
println("max: ",maximum(dtList))

std: 3221.977621410686
mean: 137.86952760681908
min: 0.0
max: 99683.71945037507


In [92]:
zeroCount=0
for time in dtList
    if time==0.0
        zeroCount+=1
    end
end
println("number of zeros in dt: ",zeroCount)
println("% zeroes: ",zeroCount/length(dtList)*100," %")
println("non-zero values: ",length(dtList)-zeroCount)

number of zeros in dt: 122511
% zeroes: 99.79635226171179 %
non-zero values: 250
